In [1]:



import open3d as o3d
from Joint import Joint
import numpy as np
# import matplotlib.pyplot as plt
from Skeleton import Skeleton
import Plotter
import plotly.graph_objects as go

# 
from os import listdir
from os.path import isfile, join
from Skin import Skin
import Plotter

mypath = 'D:/Documents/gaussian_splat/model/NeuroMechFly/data/design/meshes/stl'
path_to_mesh = 'I:/My Drive/Research/gaussian_splatting/mesh'
parts = ['body','right_wing','left_wing']

translation = [-0.1,0,1]

skin = Skin(parts,path_to_mesh,scale = 1000)
skin.translate_ptcloud_skin(translation)



Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
from Joint import Joint
import numpy as np
# import matplotlib.pyplot as plt
from Skeleton import Skeleton
import Plotter
import plotly.graph_objects as go

# %matplotlib qt

pitch_body = 0
joints = {'root_no_bone':[0,0,0],
          'neck':[0.6,0,0.3],
          'neck_thorax': [0.6,0,0.3],
          'head':[0.3,0,0],
          'thorax':[-1,0,0],
          'abdomen':[-1.3,0,0] ,
          'right_sp_no_bone': [-0,0,0.3],
          'right_wing_root':[0,-0.3,0],
          'right_wing_tip':[0,-2.2,0],
          'left_sp_no_bone':[0,0,0.3],
          'left_wing_root':[0,0.3,0],
          'left_wing_tip':[0,2.2,0]
          }

local_rotation = {'root_no_bone':[0,-pitch_body,0],
          'neck':[0,pitch_body,0],
          'neck_thorax': [0,-25,0],
          'head':[0,0,0],
          'thorax':[0,25,0],
          'abdomen':[0,0,0] ,
          'right_sp_no_bone': [0,pitch_body,0],
          'right_wing_root':[0,0,0],
          'right_wing_tip':[0,0,0],
          'left_sp_no_bone':[0,pitch_body,0],
          'left_wing_root':[-0,0,0],
          'left_wing_tip':[0,0,0]
          }

parent_child = {'root_no_bone':['neck','neck_thorax','right_sp_no_bone','left_sp_no_bone'],
                      'neck':['head'],
                      'neck_thorax':['thorax'],
                      'thorax':['abdomen'],
                      'right_sp_no_bone':['right_wing_root'],
                      'right_wing_root': ['right_wing_tip'],
                      'left_sp_no_bone': ['left_wing_root'],
                      'left_wing_root':['left_wing_tip']}

skeleton = Skeleton()
skeleton.initilize_joints(joints)
skeleton.connect_parent_child(parent_child)

skeleton.update_local_roattion(local_rotation)
skeleton.update_global_rotation(rest_bind = True)
skeleton.get_global_point_skeleton_branch()
skeleton.add_bone_to_joint()

skin.calculate_weights(skeleton,constant_weight = [None,'right_wing_root','left_wing_root'],th = 10)




In [13]:
skin.ptcloud_skin.shape

(757106, 3)

In [12]:
skin.weight.shape

(757106, 5)

In [29]:
pitch_body=40
local_rotation = {'root_no_bone':[0,-pitch_body,0],
          'neck':[0,0,0],
          'neck_thorax': [0,-25,0],
          'head':[0,0,0],
          'thorax':[0,25,0],
          'abdomen':[0,-0,0] ,
          'right_sp_no_bone': [0,pitch_body,0],
          'right_wing_root':[0,0,0],
          'right_wing_tip':[0,0,0],
          'left_sp_no_bone':[0,pitch_body,0],
          'left_wing_root':[0,0,0],
          'left_wing_tip':[0,0,0]
          }


# for joint in skeleton.joints.values():     
#         if joint.bone != None:    
        #     print(joint.bone.bone_points)
        # print(joint.global_transformation)
skeleton.update_local_roattion(local_rotation)
skeleton.update_global_rotation()
skeleton.get_global_point_skeleton_branch()
# skeleton.add_bone_to_joint()

for joint in skeleton.joints.values():     
    if joint.bone != None:    
        points_homo = np.hstack((skeleton.joints[joint.name].bone.bone_points,np.array([[1,1]]).T))
        transformation_rest = np.linalg.inv(skeleton.joints[joint.name].bind_transformation)
        rotated_points = np.dot(transformation_rest,points_homo.T)
        skeleton.joints[joint.name].bone.bone_points =  np.dot(skeleton.joints[joint.name].global_transformation,rotated_points).T[:,0:3]
        
        # joint.bone.direction = joint.rotate_to_new_position(1,np.append(joint.bone.direction,1))[0:3]
        # joint.bone.bone_points = joint.rotate_to_new_position(1,np.hstack((joint.bone.bone_points,np.array([[1,1]]).T)))[:,0:3]
        print(points_homo)
        # print(joint.bone.bone_points)
        # print(points_homo)


# points = skin.ptcloud_skin
# points_homo = np.column_stack([points,points[:,0]*0+1])


[[-0.04019238  0.         -0.66961524  1.        ]
 [-0.19019238  0.         -0.92942286  1.        ]]
[[-0.04019238  0.         -0.66961524  1.        ]
 [ 0.04696336  0.          0.32657946  1.        ]]
[[0.04696336 0.         0.32657946 1.        ]
 [0.69696336 0.         1.45241248 1.        ]]
[[-1.17786897 -0.3        -0.04097293  1.        ]
 [-1.17786897 -2.5        -0.04097293  1.        ]]
[[-1.17786897  0.3        -0.04097293  1.        ]
 [-1.17786897  2.5        -0.04097293  1.        ]]


In [30]:
skeleton.joints[joint.name].bind_transformation

array([[1. , 0. , 0. , 0. ],
       [0. , 1. , 0. , 2.5],
       [0. , 0. , 1. , 0.3],
       [0. , 0. , 0. , 1. ]])

In [30]:

marker_dict_skin = {'size':4,'color':skin.weight[::10,0],  # Set color to distances
                'colorscale':'Viridis','opacity':0.3}

marker_dict_skeleton = {'size': 10}
line_dict_skeleton = {'width': 10}
fig = go.Figure()

skin.plot_skin(fig,'skin',skip = 10,marker_dict = marker_dict_skin)
skeleton.plot_skeleton(fig,marker_dict_skeleton,line_dict_skeleton)
fig.show()


In [26]:

import plotly.graph_objects as go
import numpy as np

from plotly.subplots import make_subplots

import Plotter
points = skin.ptcloud_skin[::10,:]
pitch_body=0
local_rotation = {'root_no_bone':[0,-pitch_body,20],
          'neck':[0,0,0],
          'neck_thorax': [0,-25,0],
          'head':[0,0,0],
          'thorax':[0,25,0],
          'abdomen':[0,-0,0] ,
          'right_sp_no_bone': [0,pitch_body,0],
          'right_wing_root':[0,0,0],
          'right_wing_tip':[0,0,0],
          'left_sp_no_bone':[0,pitch_body,0],
          'left_wing_root':[0,0,0],
          'left_wing_tip':[0,0,0]
          }
skeleton.update_local_roattion(local_rotation)
skeleton.update_global_rotation()
skeleton.get_global_point_skeleton_branch()
skeleton.add_bone_to_joint()




points_homo = np.column_stack([points,points[:,0]*0+1])


idx = 0
rotated_points_per_bone = points_homo*0
for joint_name in skeleton.joints.keys():
     if skeleton.joints[joint_name].bone is not None:
        
        
        # [skeleton.joints[joint_name].rotate_to_new_position(weight[:,idx:idx + 1],points_homo) for joint_name in skeleton.joints.keys() if skeleton.joints[joint_name].bone is not None]

        transformation_rest = np.linalg.inv(skeleton.joints[joint_name].bind_transformation)
        rotated_points = np.dot(transformation_rest,points_homo.T)
        rotated_points_per_bone +=  skin.weight[::10,idx:idx + 1]*np.dot(skeleton.joints[joint_name].global_transformation,rotated_points).T
        idx += 1
rotated_points_per_bone


array([[-0.31897284,  0.04370395,  0.235494  ,  1.        ],
       [-0.37353335, -0.01589741,  0.26317774,  1.        ],
       [-0.28442915,  0.24834745,  0.16488848,  1.        ],
       ...,
       [-0.0474212 ,  2.32181144,  1.14415993,  1.        ],
       [-0.04660189,  2.31800402,  1.14295539,  1.        ],
       [-0.0443925 ,  2.31654753,  1.14398488,  1.        ]])

In [8]:
skin.weight.shape

(757106, 5)

In [18]:

import plotly.graph_objects as go
import numpy as np
from plotly.subplots import make_subplots

import Plotter




fig = go.Figure()
marker_dict = {'color':'black','size':3,'opacity':0.5}
Plotter.scatter3d(fig,rotated_points_per_bone[::1,:],'fly',marker_dict = marker_dict)


line_dict = {'width': 5,'color': 'green'}
marker_dict = {'color':'green','size':10}

for joint_name in skeleton.joints.keys():
    if 'right' in joint_name: 
        marker_dict['color'] = 'red'
        line_dict['color'] = 'red'

    elif 'left' in joint_name: 
        marker_dict['color'] = 'blue'
        line_dict['color'] = 'blue'

    else:
        marker_dict['color'] = 'green'
        line_dict['color'] = 'green'




    bone = skeleton.joints[joint_name].bone
    if bone is not None:
        Plotter.scatter3d(fig,bone.bone_points,bone.bone_points_names[0],mode = 'lines',line_dict= line_dict)
        Plotter.scatter3d(fig,bone.bone_points,bone.bone_points_names[0],mode = 'markers',marker_dict= marker_dict)

fig.show()

In [21]:
bones = [skeleton.joints[joint_name].bone for joint_name in skeleton.joints.keys() if skeleton.joints[joint_name].bone != None]
bones[4].bone_points_names

array([[ 0. , -0.4,  0.3],
       [ 0. , -2.6,  0.3]])

In [5]:
(skeleton.joints[joint_name].parent.global_origin - skeleton.joints[joint_name].global_origin)/np.abs(np.sum(skeleton.joints[joint_name].parent.global_origin - skeleton.joints[joint_name].global_origin))

array([0., 1., 0.])

In [2]:
from Bone import Bone
joint_name = 'right_wing_root'

bones = Bone(skeleton.joints[joint_name].parent,skeleton.joints[joint_name])
bones.direction

array([0., 1., 0.])

In [38]:



skeleton.joints[joint_name].global_origin - skeleton.joints[joint_name].parent.global_origin


array([ 0. , -0.4,  0. ])

In [36]:
for branch in skeleton.hirarchy:
    for idx,joint_name in enumerate(branch): 
        if (len(joint_name.split('no_bone')) < 2) & len(skeleton.joints[joint_name].child) > 0:
            print(joint_name,branch[idx + 1])

neck head
neck_thorax thorax
thorax abdomen
right_sp right_wing_root
right_wing_root right_wing_tip
left_sp left_wing_root
left_wing_root left_wing_tip


In [4]:
skeleton.joints['right_wing_root'].parent

In [9]:
skeleton.joints['root_virtual'].child

In [16]:
skeleton.joints['right_wing_root'].global_origin 
skeleton.joints['right_wing_root'].child[0].global_origin 

bone = []
joint = skeleton.joints['root_no_bone']
for child in joint.child:
        if joint.joint_without_bone == False:
                bone = (joint.global_origin - child.global_origin)/np.abs(np.sum(joint.global_origin - child.global_origin))
bone


[]

In [27]:
branch = skeleton.hirarchy[0]
branch[0].split('no_bone')

['root_', '']

In [20]:
branch = skeleton.hirarchy[0]
joint_name.split['no_bone'] for joint_name in  branch 

False

In [31]:

def get_global_transformation( joint,rest_bind = False):
    if joint.parent == None:
        return []


    joint.parent.get_global_transformation()
    print([joint.parent.name,joint.name])



get_global_transformation( skeleton.joints['right_wing_tip'],rest_bind = False)

['right_wing_root', 'right_wing_tip']


In [18]:
for branch in skeleton.hirarchy:
    
        

[['root_no_bone', 'neck', 'head'],
 ['root_no_bone', 'neck_thorax', 'thorax', 'abdomen'],
 ['root_no_bone', 'right_sp', 'right_wing_root', 'right_wing_tip'],
 ['root_no_bone', 'left_sp', 'left_wing_root', 'left_wing_tip']]

In [11]:
import plotly



import plotly.graph_objects as go
import plotly.io as pio
import plotly.express as px
import matplotlib.cm
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from plotly.subplots import make_subplots

import Plotter



points = np.asarray(mesh_list[1].vertices)
points = np.asarray(mesh_list[1].vertices)[::10,:]*1000
fig = go.Figure()

points[:,2] = points[:,2] - 1
points[:,0] = points[:,0] + 0.1

marker_dict = {'color':'black','size':3,'opacity':0.5}
Plotter.scatter3d(fig,points[::10,:],'fly', show_colorbar=False,marker_dict = marker_dict)


line_dict = {'width': 5,'color': 'green'}
marker_dict = {'color':'green','size':10}


Plotter.scatter3d(fig,all_points_per_branch[0][1:],'head_nech',mode = 'lines',line_dict= line_dict, show_colorbar=False)
Plotter.scatter3d(fig,all_points_per_branch[1][1:],'neck_thorax',mode = 'lines',line_dict= line_dict, show_colorbar=False)

Plotter.scatter3d(fig,all_points_per_branch[0][1:],'head_nech',mode = 'markers', show_colorbar=False,marker_dict = marker_dict)
Plotter.scatter3d(fig,all_points_per_branch[1][1:],'neck_thorax',mode = 'markers', show_colorbar=False,marker_dict = marker_dict)


line_dict_rw = {'width': 5,'color': 'red'}
marker_dict_rw = {'color':'red','size':10}


line_dict_lw = {'width': 5,'color': 'blue'}
marker_dict_lw = {'color':'blue','size':10}


Plotter.scatter3d(fig,all_points_per_branch[2][1:],'right wing',mode = 'lines',line_dict= line_dict_rw, show_colorbar=False)
Plotter.scatter3d(fig,all_points_per_branch[3][1:],'left wing',mode = 'lines',line_dict = line_dict_lw, show_colorbar=False)

Plotter.scatter3d(fig,all_points_per_branch[2][1:],'right wing',mode = 'markers', show_colorbar=False,marker_dict = marker_dict_rw)
Plotter.scatter3d(fig,all_points_per_branch[3][1:],'left wing',mode = 'markers', show_colorbar=False,marker_dict = marker_dict_lw)


In [10]:
all_points_per_branch[0]

array([[0. , 0. , 0. ],
       [0.1, 0. , 0. ],
       [0.2, 0. , 0. ]])

In [ ]:
points = np.asarray(mesh_list[0].vertices)*1000
fig = go.Figure()

points[:,2] = points[:,2] - 1
Plotter.scatter3d(fig,points[::10,:],[1,0,0],3,'fly')



IndexError: index 3 is out of bounds for axis 1 with size 3